# Overview

**This is a showcase how to use LangChain multi-input tools and be able to have predictable answers from the agents using these tools.**

### <font color='gray'>Loading libraries and variables</font>

In [41]:
import os
from dotenv import load_dotenv

from pydantic import BaseModel, Field
from typing import Optional, Union, List

from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import ChatPromptTemplate
from langchain.agents import AgentType
from langchain.tools import StructuredTool

In [42]:
# Loading OPENAI_API_KEY from .env file

load_dotenv()

True


### <font color='blue'>Creating LangChain custom multi-input tool</font>



In [69]:
#
# I'm creating this class to be used in args_schema parameter of my tool.
#   args_schema is a required parameter for multi-input LangChain tools and recommended for single-input tools.
#

class ContentOverviewInput(BaseModel):
    source_id: Optional[int] = Field(default=None)
    topic_id: Optional[int] = Field(default=None)



#
# My multi-input tool class.
#   Using StructuredTool is required for multi-input tools.
#

class ContentOverviewTool(StructuredTool):
    name = "Content overviewer"
    description = "use this tool when you need to overview source identified by source_id using prompts that can be extracted using topic_id"
    args_schema: ContentOverviewInput = Field(default_factory=ContentOverviewInput)


    def _run(
        self,
        topic_id: Union[int, List[int]],
        source_id: Union[int, List[int]] = None,
    ):

        
        # ...Here goes the actual code that is doing the work of the tool generating an observation
        #    (that the agent using this tool will use when generating final answer)...

        
        # For the sake of the example I'll use a hard-coded observation
        observation = "Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications."
        
        
        return observation


### <font color='gray'>Initializing</font>

In [71]:
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

# initialize conversational memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [72]:
tools = [ContentOverviewTool()]

# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)


### <font color='gray'>Calling our agent</font>

In [73]:
r = agent("Summarise source 46 using the tools from 8")



> Entering new AgentExecutor chain...
Action: 
```
{
  "action": "Content overviewer",
  "action_input": {
    "source_id": 46,
    "topic_id": 8
  }
}
```
Observation: Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications.
Thought:I need to use the content overviewer tool to summarize source 46 using the tools from topic 8. 
Action: 
```
{
  "action": "Content overviewer",
  "action_input": {
    "source_id": 46,
    "topic_id": 8
  }
}
```
Observation: Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications.
Thought:I have used the content overviewer tool to summarize source 46 using the tools from topic 8. The summary of the video is as follows: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications.

> Finished chain.



### <font color='blue'>**Good result, but...**</font>

**But what if we need the agent to give us the exact answer it received rorm the tool without additional comments of the agent?** 🤔



### <font color='blue'>Updating our tool to have more predictable answers from the agents using it</font>

The only change I make here compared to the version of ContentOverviewTool above, is adding instruction to the output of the tool.

In [74]:
class ContentOverviewTool(StructuredTool):
    name = "Content overviewer"
    description = "use this tool when you need to overview source identified by source_id using prompts that can be extracted using topic_id"
    args_schema: ContentOverviewInput = Field(default_factory=ContentOverviewInput)


    def _run(
        self,
        topic_id: Union[int, List[int]],
        source_id: Union[int, List[int]] = None,
    ):

        
        # ...Here goes the actual code that is doing the work of the tool generating an observation
        #    (that the agent using this tool will use when generating final answer)...

        
        # For the sake of the example I'll use a hard-coded observation
        observation = "Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications."

        # Adding instruction to the returned value of the tool is important
        #   if I want predictable format of the answer from the agent using this tool
        instruction = "Only answer with the summary above as is, word-for-word. Do not add comments or notes about what has been done and what is the result. Do not explain the work done or the results achieved. Only output the actual summary above and not a single character more." #"Explain this like to a 5 year old. Do not add anything else to your answer."
        
        
        return f"{observation} {instruction}"

In [75]:
tools = [ContentOverviewTool()]

# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [76]:
r = agent("Summarise source 46 using the tools from 8")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Content overviewer",
  "action_input": {
    "source_id": 46,
    "topic_id": 8
  }
}
```
Observation: Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications. Only answer with the summary above as is, word-for-word. Do not add comments or notes about what has been done and what is the result. Do not explain the work done or the results achieved. Only output the actual summary above and not a single character more.
Thought:I will use the Content overviewer tool to summarize source 46 using the prompts from topic 8.

Action:
```
{
  "action": "Content overviewer",
  "action_input": {
    "source_id": 46,
    "topic_id": 8
  }
}
```


Observation: Summary: The video discusses the concept of knowledge management and the limitations of relying solely on vector search in AI applications. Only answer with the summary above as is, word-for-word.


### <font color='blue'>Great, now my tool can control agents' answers!</font> 😈
